#Indic Trans 2 Model

which will convert eng-Indic
Indic-eng
indic-Indic

#Import file and dependencies

In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [ ]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [ ]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransToolkit.git
%cd IndicTransToolkit
!python3 -m pip install --editable ./
%cd ..

#Restart the session
Then run the cell below

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit import IndicProcessor

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [ ]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

English To Indic Translation

In [ ]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

en_sents = [
    "When I was young, I used to go to the park every day.",
    "He has many old books, which he inherited from his ancestors.",
    "I can't figure out how to solve my problem.",
    "She is very hardworking and intelligent, which is why she got all the good marks.",
    "We watched a new movie last week, which was very inspiring.",
    "If you had met me at that time, we would have gone out to eat.",
    "She went to the market with her sister to buy a new sari.",
    "Raj told me that he is going to his grandmother's house next month.",
    "All the kids were having fun at the party and were eating lots of sweets.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
]

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

tokenization_indictrans.py:   0%|          | 0.00/8.10k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json:   0%|          | 0.00/645k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(



eng_Latn - hin_Deva
eng_Latn: When I was young, I used to go to the park every day.
hin_Deva: जब मैं छोटा था, मैं हर दिन पार्क जाता था। 
eng_Latn: He has many old books, which he inherited from his ancestors.
hin_Deva: उनके पास कई पुरानी किताबें हैं, जो उन्हें अपने पूर्वजों से विरासत में मिली हैं। 
eng_Latn: I can't figure out how to solve my problem.
hin_Deva: मुझे समझ नहीं आ रहा है कि मैं अपनी समस्या का समाधान कैसे करूं। 
eng_Latn: She is very hardworking and intelligent, which is why she got all the good marks.
hin_Deva: वह बहुत मेहनती और बुद्धिमान है, यही कारण है कि उसे सभी अच्छे अंक मिले। 
eng_Latn: We watched a new movie last week, which was very inspiring.
hin_Deva: हमने पिछले हफ्ते एक नई फिल्म देखी, जो बहुत प्रेरणादायक थी। 
eng_Latn: If you had met me at that time, we would have gone out to eat.
hin_Deva: अगर आप उस समय मुझसे मिलते तो हम बाहर खाना खाने जाते। 
eng_Latn: She went to the market with her sister to buy a new sari.
hin_Deva: वह अपनी बहन के साथ नई साड़ी खरीदने के लिए 

Indic To English Translation

In [ ]:
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

hi_sents = [
    "जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।",
    "उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।",
    "मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।",
    "वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।",
    "हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।",
    "अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।",
    "वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।",
    "राज ने मुझसे कहा कि वह अगले महीने अपनी नानी के घर जा रहा है।",
    "सभी बच्चे पार्टी में मज़ा कर रहे थे और खूब सारी मिठाइयाँ खा रहे थे।",
    "मेरे मित्र ने मुझे उसके जन्मदिन की पार्टी में बुलाया है, और मैं उसे एक तोहफा दूंगा।",
]
src_lang, tgt_lang = "hin_Deva", "eng_Latn"
en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)


print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, en_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del indic_en_tokenizer, indic_en_model

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

tokenization_indictrans.py:   0%|          | 0.00/8.10k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

dict.TGT.json:   0%|          | 0.00/645k [00:00<?, ?B/s]

...

modeling_indictrans.py:   0%|          | 0.00/79.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]


hin_Deva - eng_Latn
hin_Deva: जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।
eng_Latn: When I was young, I used to go to the park every day.
hin_Deva: उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।
eng_Latn: She has a lot of old books, which she inherited from her grandparents.
hin_Deva: मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।
eng_Latn: I don't know how to find a solution to my problem.
hin_Deva: वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।
eng_Latn: He is very hardworking and understanding, so he got all the good marks.
hin_Deva: हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।
eng_Latn: We saw a new movie last week that was very inspiring.
hin_Deva: अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।
eng_Latn: If you'd given me a pass at that time, we'd have gone out to eat.
hin_Deva: वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।
eng_Latn: She had gone to the market wit

Indic To Indic Translation

In [ ]:
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-indic-indic-dist-320M
indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

hi_sents = [
    "जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।",
    "उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।",
    "मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।",
    "वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।",
    "हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।",
    "अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।",
    "वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।",
    "राज ने मुझसे कहा कि वह अगले महीने अपनी नानी के घर जा रहा है।",
    "सभी बच्चे पार्टी में मज़ा कर रहे थे और खूब सारी मिठाइयाँ खा रहे थे।",
    "मेरे मित्र ने मुझे उसके जन्मदिन की पार्टी में बुलाया है, और मैं उसे एक तोहफा दूंगा।",
]
src_lang, tgt_lang = "hin_Deva", "mar_Deva"
mr_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, mr_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del indic_indic_tokenizer, indic_indic_model

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

tokenization_indictrans.py:   0%|          | 0.00/8.10k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

dict.TGT.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

...

modeling_indictrans.py:   0%|          | 0.00/79.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]


hin_Deva - mar_Deva
hin_Deva: जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।
mar_Deva: मी लहान होतो तेव्हा मी दररोज उद्यानाला जायचे. 
hin_Deva: उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।
mar_Deva: तिच्याकडे बरीच जुनी पुस्तके आहेत, जी तिला तिच्या आजोबांकडून वारशाने मिळाली आहेत. 
hin_Deva: मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।
mar_Deva: माझ्या समस्येवर तोडगा कसा काढायचा हे मला समजत नाही. 
hin_Deva: वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।
mar_Deva: तो खूप मेहनती आणि बुद्धिमान आहे, त्यामुळे त्याला सर्व चांगले गुण मिळाले. 
hin_Deva: हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।
mar_Deva: आम्ही गेल्या आठवड्यात एक नवीन चित्रपट पाहिला जो खूप प्रेरणादायी होता. 
hin_Deva: अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।
mar_Deva: जर तुम्हाला त्या वेळी मला पास मिळाला तर आम्ही बाहेर जेवायला जाऊ. 
hin_Deva: वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।
mar_Deva: ती ति

Running the code with user input

# User input
user will give input in text format

In [ ]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

def get_user_input():
    user_input = input("Enter the text you want to translate: ")
    return [user_input]

en_sents = get_user_input()

src_lang, tgt_lang = "eng_Latn", "ory_Orya"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


output_file_path = 'translated_text.txt'

with open(output_file_path, 'w', encoding='utf-8') as file:
    for input_sentence, translation in zip(en_sents, hi_translations):
        file.write(f"{src_lang}: {input_sentence}\n")
        file.write(f"{tgt_lang}: {translation}\n\n")

print(f"Translations have been saved to {output_file_path}")

from google.colab import files
files.download('translated_text.txt')



Enter the text you want to translate: how are you

eng_Latn - ory_Orya
eng_Latn: how are you
ory_Orya: ଆପଣ କିପରି ଅଛନ୍ତି? 
Translations have been saved to translated_text.txt


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# User File Input
user will give input in file.txt format

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    file_path = filename

with open(file_path, 'r', encoding='utf-8') as file:
    en_sents = [line.strip() for line in file.readlines()]

print("Original sentences:")
print(en_sents)

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

output_file_path = 'translated_text.txt'

with open(output_file_path, 'w', encoding='utf-8') as file:
    for input_sentence, translation in zip(en_sents, hi_translations):
        file.write(f"{src_lang}: {input_sentence}\n")
        file.write(f"{tgt_lang}: {translation}\n\n")

print(f"Translations have been saved to {output_file_path}")

files.download(output_file_path)



Saving gaurav sir.txt to gaurav sir.txt
Original sentences:
['I consent to Trust Bank collecting, using, sharing, and processing my personal information for the specified purposes. I have reviewed the information collected, understood the consent purpose, term & conditions, and privacy policy.I confirm that my consent is given freely, specifically for the stated purposes, based on a clear understanding of how my data will be used, and without any conditions.', '', 'I can withdraw my consent for processing of my personal data at any time, by accessing preference center or consent manager. If I do so, my personal data will be erased, unless there is any legal requirement to retain it.', '', 'I understand that my data will be processed according to the Digital Personal Data Protection Act, 2023. I declare that the information I provided is accurate and true.']
Translations have been saved to translated_text.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Here is the list of languages supported by the IndicTrans2 models:

Assamese (asm_Beng)  	Kashmiri (Arabic) (kas_Arab)	Punjabi (pan_Guru)
Bengali (ben_Beng)	Kashmiri (Devanagari) (kas_Deva)
Sanskrit (san_Deva)
Bodo (brx_Deva)	Maithili (mai_Deva)	Santali (sat_Olck)
Dogri (doi_Deva)	Malayalam (mal_Mlym)	Sindhi (Arabic) (snd_Arab)
English (eng_Latn)	Marathi (mar_Deva)	Sindhi (Devanagari) (snd_Deva)
Konkani (gom_Deva)	Manipuri (Bengali) (mni_Beng)	Tamil (tam_Taml)
Gujarati (guj_Gujr)	Manipuri (Meitei) (mni_Mtei)	Telugu (tel_Telu)
Hindi (hin_Deva)	Nepali (npi_Deva)	Urdu (urd_Arab)
Kannada (kan_Knda)	Odia (ory_Orya)

In [ ]:
languages_scripts = {
    "Assamese": "asm_Beng",
    "Kashmiri (Arabic)": "kas_Arab",
    "Punjabi": "pan_Guru",
    "Bengali": "ben_Beng",
    "Kashmiri (Devanagari)": "kas_Deva",
    "Sanskrit": "san_Deva",
    "Bodo": "brx_Deva",
    "Maithili": "mai_Deva",
    "Santali": "sat_Olck",
    "Dogri": "doi_Deva",
    "Malayalam": "mal_Mlym",
    "Sindhi (Arabic)": "snd_Arab",
    "English": "eng_Latn",
    "Marathi": "mar_Deva",
    "Sindhi (Devanagari)": "snd_Deva",
    "Konkani": "gom_Deva",
    "Manipuri (Bengali)": "mni_Beng",
    "Tamil": "tam_Taml",
    "Gujarati": "guj_Gujr",
    "Manipuri (Meitei)": "mni_Mtei",
    "Telugu": "tel_Telu",
    "Hindi": "hin_Deva",
    "Nepali": "npi_Deva",
    "Urdu": "urd_Arab",
    "Kannada": "kan_Knda",
    "Odia": "ory_Orya"
}

print(languages_scripts)

{'Assamese': 'asm_Beng', 'Kashmiri (Arabic)': 'kas_Arab', 'Punjabi': 'pan_Guru', 'Bengali': 'ben_Beng', 'Kashmiri (Devanagari)': 'kas_Deva', 'Sanskrit': 'san_Deva', 'Bodo': 'brx_Deva', 'Maithili': 'mai_Deva', 'Santali': 'sat_Olck', 'Dogri': 'doi_Deva', 'Malayalam': 'mal_Mlym', 'Sindhi (Arabic)': 'snd_Arab', 'English': 'eng_Latn', 'Marathi': 'mar_Deva', 'Sindhi (Devanagari)': 'snd_Deva', 'Konkani': 'gom_Deva', 'Manipuri (Bengali)': 'mni_Beng', 'Tamil': 'tam_Taml', 'Gujarati': 'guj_Gujr', 'Manipuri (Meitei)': 'mni_Mtei', 'Telugu': 'tel_Telu', 'Hindi': 'hin_Deva', 'Nepali': 'npi_Deva', 'Urdu': 'urd_Arab', 'Kannada': 'kan_Knda', 'Odia': 'ory_Orya'}


In [ ]:
# en_indic_model.save_pretrained('./model')
# en_indic_tokenizer.save_pretrained('./tokenizer')

In [ ]:
import os
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Function to initialize the model and tokenizer
def initialize_model_and_tokenizer(ckpt_dir, quantization=None):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig is None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()
    return tokenizer, model

# Function to save the model and tokenizer
def save_model_and_tokenizer(model, tokenizer, save_directory):
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    model.save_pretrained(save_directory)
    tokenizer.save_pretrained(save_directory)
    print(f"Model and tokenizer saved to {save_directory}")

# Specify the checkpoint directory and save directory
ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # Change this to your model's directory
save_directory = "saved_model/indictrans2-en-indic"

# Initialize the model and tokenizer
tokenizer, model = initialize_model_and_tokenizer(ckpt_dir)

# Save the model and tokenizer
save_model_and_tokenizer(model, tokenizer, save_directory)


The repository for ai4bharat/indictrans2-en-indic-1B contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/ai4bharat/indictrans2-en-indic-1B.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
Model and tokenizer saved to saved_model/indictrans2-en-indic


In [ ]:
# import os
# import shutil
# from flask import Flask, send_file

# app = Flask(__name__)

# def create_zip_file(directory):
#     shutil.make_archive(directory, 'zip', directory)

# @app.route('/download_model')
# def download_model():
#     model_directory = 'saved_model/indictrans2-en-indic'
#     zip_filename = 'saved_model/indictrans2-en-indic.zip'

#     # Create a zip file of the model directory
#     create_zip_file(model_directory)

#     # Send the zip file as a download response
#     return send_file(zip_filename, as_attachment=True)

# if __name__ == '__main__':
#     app.run(host='0.0.0.0', port=5000)


In [ ]:
import pickle

In [ ]:
pickle.dump(model, open("/content/saved_model", "wb"))

In [ ]:
from google.colab import files
import shutil

# Path to your saved model directory
model_directory = 'saved_model/indictrans2-en-indic'

# Create a zip file of the model directory
shutil.make_archive(model_directory, 'zip', model_directory)

# Download the zip file
files.download(f'{model_directory}.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>